In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/formula1/races.csv
/kaggle/input/formula1/constructor_results.csv
/kaggle/input/formula1/drivers.csv
/kaggle/input/formula1/constructors.csv
/kaggle/input/formula1/lap_times.csv
/kaggle/input/formula1/status.csv
/kaggle/input/formula1/driver_standings.csv
/kaggle/input/formula1/seasons.csv
/kaggle/input/formula1/pit_stops.csv
/kaggle/input/formula1/constructor_standings.csv
/kaggle/input/formula1/results.csv
/kaggle/input/formula1/circuits.csv
/kaggle/input/formula1/qualifying.csv


In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import re
import datetime

In [3]:
path="/kaggle/input/formula1/"
constructors_raw=pd.read_csv(path+"constructors.csv")
constructors_st_raw=pd.read_csv(path+"constructor_standings.csv")
constructors_res_raw=pd.read_csv(path+"constructor_results.csv")
drivers_raw=pd.read_csv(path+"drivers.csv")
driver_st_raw=pd.read_csv(path+"driver_standings.csv")
results_raw=pd.read_csv(path+"results.csv")
status=pd.read_csv(path+"status.csv")
races_raw=pd.read_csv(path+"races.csv")
pit_stops_raw=pd.read_csv(path+"pit_stops.csv")
quali_raw=pd.read_csv(path+"qualifying.csv")
lap_times_raw=pd.read_csv(path+"lap_times.csv")

In [4]:
races_raw=races_raw[races_raw["raceId"]>=841]
races_raw=races_raw.sort_values(["raceId"],ascending=True)
results_raw=results_raw[results_raw["raceId"]>=841]
results_raw=results_raw.sort_values(["raceId"],ascending=True)
races_raw_int=results_raw.merge(status,how='outer')
races_raw_int.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,status
0,20779.0,841.0,20.0,9.0,1,1.0,1,1,1.0,25.0,58.0,1:29:30.259,5370259,44,4,1:29.844,212.488,1,Finished
1,20781.0,841.0,808.0,4.0,10,6.0,3,3,3.0,15.0,58.0,+30.560,5400819,55,7,1:30.064,211.969,1,Finished
2,20782.0,841.0,4.0,6.0,5,5.0,4,4,4.0,12.0,58.0,+31.772,5402031,49,2,1:29.487,213.336,1,Finished
3,20783.0,841.0,17.0,9.0,2,3.0,5,5,5.0,10.0,58.0,+38.171,5408430,50,3,1:29.600,213.066,1,Finished
4,20784.0,841.0,18.0,1.0,4,4.0,6,6,6.0,8.0,58.0,+54.304,5424563,49,5,1:29.883,212.396,1,Finished


In [5]:
results=races_raw.merge(races_raw_int,how='outer')
results.columns

Index(['raceId', 'year', 'round', 'circuitId', 'name', 'date', 'race_time',
       'url', 'resultId', 'driverId', 'constructorId', 'number', 'grid',
       'position', 'positionText', 'positionOrder', 'points', 'laps', 'time',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId', 'status'],
      dtype='object')

In [6]:
results_drop=['url','resultId','positionText','statusId']
results=results.drop(columns=results_drop,axis=1)
results.head()

,raceId,year,round,circuitId,name,date,race_time,driverId,constructorId,number,...,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,status
0,841.0,2011.0,1.0,1.0,Australian Grand Prix,27-03-2011,06:00:00,20.0,9.0,1,...,1.0,25.0,58.0,1:29:30.259,5370259,44,4,1:29.844,212.488,Finished
1,841.0,2011.0,1.0,1.0,Australian Grand Prix,27-03-2011,06:00:00,808.0,4.0,10,...,3.0,15.0,58.0,+30.560,5400819,55,7,1:30.064,211.969,Finished
2,841.0,2011.0,1.0,1.0,Australian Grand Prix,27-03-2011,06:00:00,4.0,6.0,5,...,4.0,12.0,58.0,+31.772,5402031,49,2,1:29.487,213.336,Finished
3,841.0,2011.0,1.0,1.0,Australian Grand Prix,27-03-2011,06:00:00,17.0,9.0,2,...,5.0,10.0,58.0,+38.171,5408430,50,3,1:29.600,213.066,Finished
4,841.0,2011.0,1.0,1.0,Australian Grand Prix,27-03-2011,06:00:00,18.0,1.0,4,...,6.0,8.0,58.0,+54.304,5424563,49,5,1:29.883,212.396,Finished


In [7]:
col=["driverRef","number","code","url"]
drivers_raw=drivers_raw.drop(col,axis=1)
drivers_raw['dob']=pd.to_datetime(drivers_raw['dob'])
drivers_raw['Age']=2021-drivers_raw['dob'].dt.year
drivers_raw=drivers_raw.drop("dob",axis=1)
drivers_raw.shape

(853, 5)

In [8]:
# driver_index=driver_st_raw.loc[driver_st_raw["raceId"]== 841].index.min()
# driver_st_raw=driver_st_raw[driver_index:]
#driver_st_raw=driver_st_raw.sort_values(["raceId","driverId"],ascending=True)
#driver_st_raw.head(10)
# driver_st_raw.loc[driver_st_raw["raceId"]== 841].index
driver_st_raw=driver_st_raw[driver_st_raw["raceId"]>=841]
driver_st_raw=driver_st_raw.sort_values(["raceId","driverId"],ascending=True)
drop_columns=['positionText','wins','driverStandingsId']
driver_st_raw=driver_st_raw.drop(columns=drop_columns,axis=1)
driver_st_raw.head()

,driverId,raceId,points,position
28439,1,841,18.0,2
28429,2,841,0.0,12
28423,3,841,0.0,17
28437,4,841,12.0,4
28422,5,841,0.0,18


In [9]:
driver_data=drivers_raw.merge(driver_st_raw,how='outer')
driver_data.head(10)
#driver_data.shape

,driverId,forename,surname,nationality,Age,raceId,points,position
0,1,Lewis,Hamilton,British,36,841.0,18.0,2.0
1,1,Lewis,Hamilton,British,36,842.0,22.0,3.0
2,1,Lewis,Hamilton,British,36,843.0,47.0,2.0
3,1,Lewis,Hamilton,British,36,844.0,59.0,2.0
4,1,Lewis,Hamilton,British,36,845.0,77.0,2.0
5,1,Lewis,Hamilton,British,36,846.0,85.0,2.0
6,1,Lewis,Hamilton,British,36,847.0,85.0,4.0
7,1,Lewis,Hamilton,British,36,848.0,97.0,4.0
8,1,Lewis,Hamilton,British,36,849.0,109.0,4.0
9,1,Lewis,Hamilton,British,36,850.0,134.0,3.0


In [10]:
constructors_st_raw=constructors_st_raw[constructors_st_raw["raceId"]>=841]
constructors_res_raw=constructors_res_raw[constructors_res_raw["raceId"]>=841]
constructors_st_raw=constructors_st_raw.sort_values(["raceId"],ascending=True)
constructors_res_raw=constructors_res_raw.sort_values(["raceId"],ascending=True)
# print(constructors_st_raw.head(10))
# print(constructors_res_raw.head(10))
constructors_raw=constructors_raw.rename({'name':'Cons_name','nationality':'Cons_nationality'},axis=1)
constructors_raw=constructors_raw.drop(columns=['constructorRef','url'],axis=1)

In [11]:
constructors_data=constructors_raw.merge(constructors_st_raw,how='outer')
c_d=["constructorStandingsId","positionText"]
constructors_data=constructors_data.drop(columns=c_d,axis=1)
print(constructors_data.head(10))

   constructorId Cons_name Cons_nationality  raceId  points  position  wins
0              1   McLaren          British   841.0    26.0       2.0   0.0
1              1   McLaren          British   842.0    48.0       2.0   0.0
2              1   McLaren          British   843.0    85.0       2.0   1.0
3              1   McLaren          British   844.0   105.0       2.0   1.0
4              1   McLaren          British   845.0   138.0       2.0   1.0
5              1   McLaren          British   846.0   161.0       2.0   1.0
6              1   McLaren          British   847.0   186.0       2.0   2.0
7              1   McLaren          British   848.0   206.0       2.0   2.0
8              1   McLaren          British   849.0   218.0       2.0   2.0
9              1   McLaren          British   850.0   243.0       2.0   3.0


In [12]:
pit_stops_raw=pit_stops_raw.drop(["time","duration"],axis=1)
pit_stops_raw=pit_stops_raw.sort_values(["raceId","driverId"],ascending=True)
quali_raw=quali_raw.drop(["qualifyId","number"],axis=1)

# **Start of Lap_Times dataset Cleaning**

In [13]:
#lap_times_raw['count']=lap_times_raw.groupby(["raceId","driverId"])["milliseconds"].transform('count')
#print(lap_times_raw.head(80))
new_lap_times=pd.DataFrame(lap_times_raw.groupby(["raceId","driverId"]).mean(["milliseconds"]).reset_index())

In [14]:
rename1={'milliseconds':'Avg_lap_time','lap':'total_laps'}
new_lap_times=new_lap_times.rename(columns=rename1)
new_lap_times=new_lap_times.drop(['position'],axis=1)

In [15]:
race_index=new_lap_times.loc[new_lap_times["raceId"]==841].index.min()
new_lap_times=new_lap_times[race_index:]

In [16]:
print(new_lap_times.head())
new_lap_times.shape

      raceId  driverId  total_laps  Avg_lap_time
5233     841         1        29.5  92975.103448
5234     841         2        29.0  95507.035088
5235     841         3        11.5  96448.909091
5236     841         4        29.5  93138.465517
5237     841         5        10.0  98761.000000


(4057, 4)

# **End of Lap_Times dataset cleaning**

# **Start of Quali_data cleaning**

In [17]:
pit_stops_raw=pit_stops_raw.rename(columns={"lap":"laps_pit","milliseconds":"avg_pit_ms"})
quali_raw=quali_raw.rename(columns={"position":"q_pos"})
q_race_index=quali_raw.loc[quali_raw["raceId"]==841].index.min()
q_race_index

4535

In [18]:
quali_raw=quali_raw[q_race_index:]

In [19]:
quali_raw=quali_raw.sort_values(["raceId","driverId"],ascending=True)
quali_raw=quali_raw.drop(7276,axis=0)

In [20]:
# quali_raw['best_quali_time']=0
time_convert=['q1','q2','q3']
for t in time_convert:
    #quali_raw[t]=re.sub(r"\\N",'',quali_raw[t])
    #print(quali_raw['q1'].head())
    quali_raw[t]=quali_raw[t].str.replace(r'\\N','')
    #print(quali_raw.head(15))
quali_raw=quali_raw.replace(r'^\s*$',np.nan,regex=True)
#quali_raw.isnull().sum()
quali_raw.head(15)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


,raceId,driverId,constructorId,q_pos,q1,q2,q3
4536,841,1,1,2,01:25.4,01:24.6,01:24.3
4552,841,2,4,18,01:27.2,NaN,NaN
4541,841,3,131,7,01:25.9,01:25.6,01:25.4
4539,841,4,6,5,01:25.7,01:25.2,01:25.0
4553,841,5,205,19,01:29.3,NaN,NaN
4555,841,10,166,21,01:29.9,NaN,NaN
4542,841,13,6,8,01:26.0,01:25.6,01:25.6
4554,841,15,205,20,01:29.3,NaN,NaN
4550,841,16,10,16,01:26.2,01:31.4,NaN
4537,841,17,9,3,01:25.9,01:24.7,01:24.4


In [21]:
from datetime import datetime
# quali_raw['q1'].str.replace(' :',' ')
# quali_raw['q1'].str.replace('.',' ')
datetime.strptime(quali_raw.iloc[2]['q1'],'%M:%S.%f').strftime("%f")
#qulai_raw['ms']=datetime.time(quali_raw['q1'])
#quali_raw.iloc[2]['q1'].timestamp()*1000
# quali_raw['q1']=quali_raw['q1'].astype(np.int64)/int(1e6)
#datetime.time(quali_raw['q1'])
#quali_raw.groupby(["raceId","driverId"]).head()

'900000'

Modify Quali excel to have best quali time over three sessions
--convert to milliseconds
--find the lowest time

# **End of Quali_data cleaning**

# **Start of Pit_Stops_Cleaning**

In [22]:
pit_stops_raw.head(10)

,raceId,driverId,stop,laps_pit,avg_pit_ms
11,841,1,1,16,23227
33,841,1,2,36,23199
10,841,2,1,15,22994
29,841,2,2,30,25098
13,841,3,1,16,23716
3,841,4,1,12,23251
26,841,4,2,27,24733
43,841,4,3,42,24181
19,841,5,1,17,24865
21,841,10,1,18,23792


In [23]:
pit_stops_raw=pit_stops_raw.groupby(["raceId","driverId"]).agg({'stop':'count','laps_pit': list , 'avg_pit_ms':'mean'}).reset_index()
pit_filtered=pit_stops_raw.drop(columns="laps_pit",axis=1)

In [24]:
pit_filtered.head(10)

,raceId,driverId,stop,avg_pit_ms
0,841,1,2,23213.000000
1,841,2,2,24046.000000
2,841,3,1,23716.000000
3,841,4,3,24055.000000
4,841,5,1,24865.000000
5,841,10,1,23792.000000
6,841,13,3,24145.666667
7,841,15,2,24873.500000
8,841,16,2,24924.500000
9,841,17,3,24058.666667


# **End of Pit_stops cleaning**

In [25]:
print(quali_raw.head())
print(pit_filtered.head())
print(new_lap_times.head())

      raceId  driverId  constructorId  q_pos       q1       q2       q3
4536     841         1              1      2  01:25.4  01:24.6  01:24.3
4552     841         2              4     18  01:27.2      NaN      NaN
4541     841         3            131      7  01:25.9  01:25.6  01:25.4
4539     841         4              6      5  01:25.7  01:25.2  01:25.0
4553     841         5            205     19  01:29.3      NaN      NaN
   raceId  driverId  stop  avg_pit_ms
0     841         1     2     23213.0
1     841         2     2     24046.0
2     841         3     1     23716.0
3     841         4     3     24055.0
4     841         5     1     24865.0
      raceId  driverId  total_laps  Avg_lap_time
5233     841         1        29.5  92975.103448
5234     841         2        29.0  95507.035088
5235     841         3        11.5  96448.909091
5236     841         4        29.5  93138.465517
5237     841         5        10.0  98761.000000


In [26]:
race_data=new_lap_times.merge(pit_filtered,how='outer')
print(race_data.head())
print(race_data.shape)

   raceId  driverId  total_laps  Avg_lap_time  stop  avg_pit_ms
0     841         1        29.5  92975.103448   2.0     23213.0
1     841         2        29.0  95507.035088   2.0     24046.0
2     841         3        11.5  96448.909091   1.0     23716.0
3     841         4        29.5  93138.465517   3.0     24055.0
4     841         5        10.0  98761.000000   1.0     24865.0
(4057, 6)


In [27]:
#results.head()

In [28]:
weekend_data=race_data.merge(quali_raw,how='inner')
#print(weekend_data.head(30))
# print(weekend_data.shape)
# driver_data.head()
weekend_data['stop']=weekend_data['stop'].fillna(int(weekend_data['stop'].mean()))
weekend_data['avg_pit_ms']=weekend_data['avg_pit_ms'].fillna(weekend_data['avg_pit_ms'].mean())
q=['q1','q2','q3']
for a in q:
    weekend_data[a]=weekend_data[a].fillna(0)
#weekend_data.isnull().sum()

In [29]:
overall_data_a=results.merge(weekend_data,how='inner')
overall_data_b=overall_data_a.merge(drivers_raw,how='inner')
overall_data_c=overall_data_b.merge(constructors_raw,how='inner')
#overall_data_c.head()
#overall_data_c.columns

In [30]:
overall_data_c.columns

Index(['raceId', 'year', 'round', 'circuitId', 'name', 'date', 'race_time',
       'driverId', 'constructorId', 'number', 'grid', 'position',
       'positionOrder', 'points', 'laps', 'time', 'milliseconds', 'fastestLap',
       'rank', 'fastestLapTime', 'fastestLapSpeed', 'status', 'total_laps',
       'Avg_lap_time', 'stop', 'avg_pit_ms', 'q_pos', 'q1', 'q2', 'q3',
       'forename', 'surname', 'nationality', 'Age', 'Cons_name',
       'Cons_nationality'],
      dtype='object')

In [31]:
overall_data_c['Driver_name']=overall_data_c['forename']+ ' ' +overall_data_c['surname']

In [32]:
overall_drop=['circuitId','race_time','total_laps','forename','surname']
overall_data=overall_data_c.drop(columns=overall_drop,axis=1)
columns_order=['raceId','year','round','name','date','driverId','Driver_name','number','nationality','Age','constructorId','Cons_name','Cons_nationality','q_pos','q1','q2','q3','grid','positionOrder','position','points','laps','time','milliseconds','fastestLap','fastestLapTime','fastestLapSpeed','Avg_lap_time','stop','avg_pit_ms','rank','status']
overall_data=overall_data[columns_order]
overall_data.head()

,raceId,year,round,name,date,driverId,Driver_name,number,nationality,Age,...,time,milliseconds,fastestLap,fastestLapTime,fastestLapSpeed,Avg_lap_time,stop,avg_pit_ms,rank,status
0,841.0,2011.0,1.0,Australian Grand Prix,27-03-2011,20.0,Sebastian Vettel,1,German,34,...,1:29:30.259,5370259,44,1:29.844,212.488,92590.672414,2.0,23319.500000,4,Finished
1,842.0,2011.0,2.0,Malaysian Grand Prix,10-04-2011,20.0,Sebastian Vettel,1,German,34,...,1:37:39.832,5859832,33,1:41.539,196.523,104639.857143,3.0,22408.333333,6,Finished
2,843.0,2011.0,3.0,Chinese Grand Prix,17-04-2011,20.0,Sebastian Vettel,1,German,34,...,+5.198,5823424,47,1:41.321,193.677,103989.714286,2.0,21764.000000,8,Finished
3,844.0,2011.0,4.0,Turkish Grand Prix,08-05-2011,20.0,Sebastian Vettel,1,German,34,...,1:30:17.558,5417558,50,1:29.937,213.669,93406.172414,4.0,20402.250000,3,Finished
4,845.0,2011.0,5.0,Spanish Grand Prix,22-05-2011,20.0,Sebastian Vettel,1,German,34,...,1:39:03.301,5943301,60,1:27.162,192.262,90050.015152,4.0,20214.500000,4,Finished


In [33]:
overall_data.isnull().sum()
#overall_data.shape
# overall_data.tail()

raceId              0
year                0
round               0
name                0
date                0
driverId            0
Driver_name         0
number              0
nationality         0
Age                 0
constructorId       0
Cons_name           0
Cons_nationality    0
q_pos               0
q1                  0
q2                  0
q3                  0
grid                0
positionOrder       0
position            0
points              0
laps                0
time                0
milliseconds        0
fastestLap          0
fastestLapTime      0
fastestLapSpeed     0
Avg_lap_time        0
stop                0
avg_pit_ms          0
rank                0
status              0
dtype: int64